In [238]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from ipywidgets import IntProgress
from sklearn.metrics import classification_report

import numpy as np

tf.enable_v2_behavior()

In [239]:
def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def tf_gaussian_mech(v, sensitivity, epsilon, delta):
    return v + tf.random.normal(v.shape, mean=0.0, stddev=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

In [240]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [241]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [242]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(28, kernel_size=(3,3), input_shape=input_shape),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),  
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])
model.summary()

models = [model1]

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 26, 26, 28)        280       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 13, 13, 28)        0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 4732)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 128)               605824    
_________________________________________________________________
dropout_21 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 10)                1290      
Total params: 607,394
Trainable params: 607,394
Non-trainable params: 0
_______________________________________________

In [256]:
def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

class SGOptimizer(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.01, name="SGOptimizer", **kwargs):
        """Call super().__init__() and use _set_hyper() to store hyperparameters"""
        super().__init__(name, **kwargs)
        self._set_hyper("learning_rate", kwargs.get("lr", learning_rate)) # handle lr=learning_rate
        self._is_first = True
    
    def _create_slots(self, var_list):
        """For each model variable, create the optimizer variable associated with it.
        TensorFlow calls these optimizer variables "slots".
        For momentum optimization, we need one momentum slot per model variable.
        """
        for var in var_list:
            self.add_slot(var, "pv") #previous variable i.e. weight or bias
        for var in var_list:
            self.add_slot(var, "pg") #previous gradient


    @tf.function
    def _resource_apply_dense(self, grad, var):
        """Update the slots and perform one optimization step for one model variable
        """
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        
        new_var_m = var - tf_gaussian_mech(grad, 3.0/len(x_train), 2, 1e-5) * lr_t
        #new_var_m = var - grad * lr_t
        new_var = new_var_m
        var.assign(new_var)

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
        }

In [257]:
es = callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
optimizer = SGOptimizer()
model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(x=x_train,y=y_train, epochs=1, callbacks=[es], batch_size=64)

938/938 [==============================] - 12s 13ms/step - loss: 3.6066 - accuracy: 0.2339


In [202]:
tf.random.normal([4], 0, 1, tf.float32)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([-0.35595608, -0.7547564 , -0.16059794, -0.6739014 ], dtype=float32)>

In [169]:
optimizer.grad

<tf.Tensor 'grad:0' shape=(10,) dtype=float32>

In [251]:
t = tf.constant([1.0, 1.0, 1.0, 1.0])
t

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>

In [255]:
t_g = tf_gaussian_mech(t, 3.0/len(x_train), 0.1, 1e-5)
t_g

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1.0008119, 1.0006664, 0.9994978, 1.0008285], dtype=float32)>